In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pandas as pd
import data_transformation as transformer
import crewai_mapper as mapper

In [3]:
INPUT_PATH = Path("./input_data")
OUTPUT_PATH = Path("./output_data")

## BDO Employee Provides a set of rules for the Automapper: 
1. What is the expected information? Which columns should we map on?
2. What is the Excel format? Is it an SAP extract?
3. What information can be discarded for certain?

In [4]:
VALIDATION_RULES = {
                    "parametres": {
                        "names": ["Debiteurnaam", "Factuurnummer", "Datum", "Bedrag_EUR"],
                        "pandas_formatting": ["SAP"],
                        "formatting-SAP": "Cust. Name",
                        "to_discard": [
                            "Belegart",
                            "Sonderhauptb.Kennz.",
                            "Symbol Nettofälligkeit",
                            "Text",
                            "Hauswährung",
                            "Ausgleichsbeleg",
                            "Zuordnung"
                            ],
                    }
                }

## BDO Employee Provides a file to map from. In this case it's an SAP data extract, with a non-standard table format:

In [5]:
assert INPUT_PATH.exists()
df = pd.read_excel(INPUT_PATH / "Openstaande posten crediteuren 300423 SAP.XLSX")
df.head(5)

,Symbol Posten offen/ausg.,Zuordnung,Belegnummer,Belegart,Belegdatum,Sonderhauptb.Kennz.,Symbol Nettofälligkeit,Betrag in Hauswährung,Hauswährung,Ausgleichsbeleg,Text
0,NaN,6090466822,1.200402e+09,RE,2023-04-30,NaN,NaN,51671.15,EUR,NaN,NaN
1,NaN,6090466823,1.200402e+09,RE,2023-04-30,NaN,NaN,3919.22,EUR,NaN,NaN
2,@5C\Qoffen@,NaN,NaN,NaN,NaT,NaN,NaN,55590.37,EUR,NaN,NaN
3,Konto 30036349,NaN,NaN,NaN,NaT,NaN,NaN,55590.37,EUR,NaN,NaN
4,NaN,9503325616,1.200401e+09,RE,2023-04-25,NaN,NaN,-115.65,EUR,1.600014e+09,NaN


In [7]:
DESTINATION_COLUMNS = VALIDATION_RULES["parametres"]["names"]
SOURCE_COLUMNS = df.columns.to_list()
TO_DISCARD = VALIDATION_RULES["parametres"]["to_discard"]

In [8]:
sub_df = transformer.transform_nonstandard_SAP(df=df, new_column_name="Cust. name")

In [9]:
csv =sub_df.head(10).to_csv(sep=',')

In [ ]:
{"cust.name": "text"}

In [10]:
response = mapper.get_mapping(expected=DESTINATION_COLUMNS, input=csv)



> Entering new CrewAgentExecutor chain...
I need to understand what are the meaning of the columns in the dataset. The input data seems to be in German, but the expected columns are in Dutch. 

1. 'Debiteurnaam' and 'Cust. name' seems to be the same thing. The first is in Dutch and the second in English. 

2. 'Factuurnummer' = 'Belegnummer'

3. 'Datum' = 'Belegdatum'

4. 'Bedrag_EUR' is related to 'Betrag in Hauswährung' but I need to understand if there is any transformation or just a name difference.

Final Answer:

Cust. name --> dataset_column:  Text

Belegnummer --> dataset_column:  Factuurnummer

Belegdatum --> dataset_column:  Belegdatum

Betrag in Hauswährung --> dataset_column:  Bedrag_EUR

Note: the name of the dataset columns are in German, but I used the English translation for the sake of the task. 

I decided to return the actual complete content as the final answer, as requested. But I think it would be more useful to return a summary, in case of a large dataset. I cou

In [15]:
source_columns = transformer.extract_substrings(response, SOURCE_COLUMNS)
destination_columns = transformer.extract_substrings(response, DESTINATION_COLUMNS)

In [12]:
mapping = dict(zip(source_columns, destination_columns))

In [13]:
try:
    df = df.rename(columns=mapping)
    for col in TO_DISCARD:
        try:
            df = df.drop(columns=[col])
        except KeyError:
            print(f"Column '{col}' not found in the DataFrame and will be skipped.")
except KeyError as e:
    print(f"Error: {e}")
    print("Please check the column names in the mapping dictionary and to_discard list.")

Column 'Text' not found in the DataFrame and will be skipped.


In [14]:
df

,Symbol Posten offen/ausg.,Bedrag_EUR,Factuurnummer,Datum,Factuurnummer
0,NaN,1.200402e+09,2023-04-30,51671.15,NaN
1,NaN,1.200402e+09,2023-04-30,3919.22,NaN
2,@5C\Qoffen@,NaN,NaT,55590.37,NaN
3,Konto 30036349,NaN,NaT,55590.37,NaN
4,NaN,1.200401e+09,2023-04-25,-115.65,NaN
...,...,...,...,...,...
683,Konto 30139870,NaN,NaT,-165902.22,NaN
684,NaN,1.100129e+09,2023-04-12,-92.56,Jumbo-1283362112 keuken/kantoor
685,@5C\Qoffen@,NaN,NaT,-92.56,NaN
686,Konto 30140793,NaN,NaT,-92.56,NaN
